# 0. Load model and dataset

In [1]:
import torch, os, rasterio, skimage
from addict import Dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from ptflops import get_model_complexity_info

from dataset import HySpecNet11k
from SwinCom import SpectralGroupSwinUNet

def Entropy(inputs):
    _, counts = np.unique(inputs, return_counts=True)
    p = counts/sum(counts)
    entropy = -sum(p*np.log2(p))
    return entropy

def fit(x, y):
    c, h, w = x.shape
    y, x = y.transpose(1, 2, 0).reshape(-1, c), x.transpose(1, 2, 0).reshape(-1, c)
    mean_y, mean_x = y.mean(), x.mean()
    y, x = y - mean_y, x - mean_x
    
    beta = np.linalg.pinv(x.T @ x) @ x.T @ y
    y_ = np.round((x @ beta) + mean_y)
    y_ = y_.reshape(h, w, c).transpose(2, 0, 1).astype(np.int16)
    return y_, beta

/home/deic/miniconda3/envs/zuoxin/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/deic/miniconda3/envs/zuoxin/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2025-07-11 11:57:12.977201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-11 11:57:13.000727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-11 11:57:13.000745: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc

In [2]:
bit_shift=10

res = pd.DataFrame([])
with torch.no_grad():
    for bit_shift in [6, 7, 8, 9, 10]:
        model = SpectralGroupSwinUNet().from_pretrained(f'PreTrain/SGSU_K={bit_shift}').to('cpu')
        test_set = HySpecNet11k('/home/deic/zuoxin/Dataset/HySpecNet11k', 'easy', mode="test", bit_shift=bit_shift, is_total=False)
        psnr, ssim, Entropy_img, Entropy_MSB, Entropy_residual, Entropy_residual2 = [], [], [], [], [], []
        for file in test_set.files:
            img, msb = test_set.load_data(file)
            out = torch.clamp(model(test_set.prepare_msb(msb).unsqueeze(0)), 0.0, 1.0)
            out = np.round(out.squeeze().detach().numpy() * 10000).astype(np.int16)
            out, beta = fit(out, img)
            max_value = np.max([out.max(), img.max()])
            psnr.append(skimage.metrics.peak_signal_noise_ratio(out, img, data_range=max_value))
            ssim.append(skimage.metrics.structural_similarity(out, img, data_range=max_value))
            residual = img - out
            residual = np.concatenate([residual[0][None, :], residual[1:]-residual[:-1]], axis=0)
            msb = img >> bit_shift
            msb = np.concatenate([msb[0][None, :], msb[1:]-msb[:-1]], axis=0)
            Entropy_MSB.append(Entropy(msb))
            Entropy_residual.append(Entropy(residual))
            
        metrix = dict(BitShift=int(bit_shift), PSNR=np.mean(psnr), SSIM=np.mean(ssim), 
                      Entropy_MSB=np.mean(Entropy_MSB), Entropy_residual=np.mean(Entropy_residual))
        res = pd.concat([res, pd.Series(metrix)], axis=1)
res = res.T
res = res.reset_index(drop=True)
res

/home/deic/miniconda3/envs/zuoxin/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


,BitShift,PSNR,SSIM,Entropy_MSB,Entropy_residual
0,6.0,45.280275,0.992801,2.171043,6.371312
1,7.0,44.883903,0.992062,1.488202,6.376611
2,8.0,43.726225,0.989210,0.955253,6.405887
3,9.0,40.922772,0.977838,0.557075,6.475108
4,10.0,38.579958,0.963324,0.340648,6.529373


In [3]:
macs, params = get_model_complexity_info(
    model, (202, 128, 128), 
    as_strings=True,
    print_per_layer_stat=False, 
    verbose=False)
print(f'FLOPs: {macs}')
print(f'Params: {params}')

FLOPs: 13.62 GMac
Params: 32.56 M
